In [1]:
from __future__ import division

import os
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

import torch
from torch.nn import *
import torch.nn.functional as F
from torch import optim
import torchvision.models as models
from torch.utils.data import DataLoader

from dataloaderraw import ImageSpecDataset
from models.model import ImgSpecModel

In [2]:
def clip_gradient(optimizer, grad_clip=0.1):
    for group in optimizer.param_groups:
        for param in group['params']:
            param.grad.data.clamp_(-grad_clip, grad_clip)

In [3]:
def set_lr(onptimizer, lr):
    for group in optimizer.parma_groups:
        group['lr'] = lr

In [4]:
def update_lr(epoch, optimizer):
    learning_rate_decay_start = 1
    learning_rate_decay_every = 3
    learning_rate_decay_rate = 0.8
    learning_rate = 4e-4
    if epoch > learning_rate_decay_start and learning_rate_decay_start >= 0:
        frac = (epoch - learning_rate_decay_start) // learning_rate_decay_every
        decay_factor = learning_rate_decay_rate  ** frac
        current_lr = learning_rate * decay_factor
        set_lr(optimizer, current_lr)

In [5]:
def save_model_latest(model, optimizer, epoch, checkpoint_path='save'):
    torch.save(model.state_dict(), os.path.join(checkpoint_path, 'model_{}.pth'.format(epoch)))
    torch.save(model.state_dict(), os.path.join(checkpoint_path, 'model_latest.pth'))
    torch.save(optimizer.state_dict(), os.path.join(checkpoint_path, 'optimizer_{}.pth'.format(epoch)))
    torch.save(optimizer.state_dict(), os.path.join(checkpoint_path, 'optimizer_latest.pth'))

In [6]:
imgspec_model = ImgSpecModel()

train_imgspec_dataset = ImageSpecDataset(root_dir='C:\\Users\\Riddhi\\project_final\\small_dataset_objects', split='train')
train_dataloader = DataLoader(train_imgspec_dataset, batch_size=10, shuffle=True, num_workers=1)

val_imgspec_dataset = ImageSpecDataset(root_dir='C:\\Users\\Riddhi\\project_final\\small_dataset_objects', split='val')
val_dataloader = DataLoader(val_imgspec_dataset, batch_size=10, shuffle=True, num_workers=4)

class_weight = torch.from_numpy(np.array([1.0, 3.0])).float()
cce_loss = CrossEntropyLoss(weight=class_weight)
optimizer = optim.SGD(imgspec_model.parameters(), lr=0.01, momentum=0.9)


In [7]:
best_val_acc = 0
print_loss_every = 50
for epoch in range(50):
    imgspec_model.train()
    for i_batch, batch in enumerate(train_dataloader):
        optimizer.zero_grad()
        img_feats = batch['img_feats'].float()
        spec_imgs = batch['spec'].unsqueeze(1).float()
        scores = batch['score'].long()
        preds = imgspec_model(img_feats, spec_imgs)
        loss = cce_loss(preds, scores)
        loss.backward()
        clip_gradient(optimizer)
        optimizer.step()
        if i_batch%print_loss_every == 0:
            print('Iteration: {}, Loss: {}'.format(i_batch, loss.item()))
    update_lr(epoch, optimizer)
    save_model_latest(imgspec_model, optimizer, epoch)
    imgspec_model.eval()
    correct_class, incorrect_class = 0, 0
    all_preds, all_org = [], []
    for i_batch, batch in enumerate(val_dataloader):
        img_feats = batch['img_feats'].float()
        spec_imgs = batch['spec'].unsqueeze(1).float()
        scores = batch['score'].float().numpy()
        preds = F.softmax(imgspec_model(img_feats, spec_imgs).detach(), dim=1)
        _, preds = torch.max(preds, 1)
        preds = preds.cpu().numpy()
        #preds = np.where(preds > 0.5, 1, 0)
        for org, pred in zip(scores, preds):
            correct_class += int(org == pred)
            incorrect_class += int(org != pred)
            all_org.append(org)
            all_preds.append(pred)
    #acc = float(correct_class) / float(correct_class + incorrect_class)
    print('\n\n')
    print(classification_report(all_org, all_preds))
    acc = f1_score(all_org, all_preds, average='macro')
    print('Epoch: {}, ValF1: {}'.format(epoch, acc))
    if acc > best_val_acc:
        print('Epoch: {}, saving better model!'.format(epoch))
        best_val_acc = acc
        save_model_best(imgspec_model, optimizer, epoch, best_val_acc, 'acc')
    print('\n\n')
        

Iteration: 0, Loss: 0.578986406326294
Iteration: 50, Loss: 1.2038652896881104
Iteration: 100, Loss: 0.6777559518814087
Iteration: 150, Loss: 0.6610709428787231
Iteration: 200, Loss: 0.9312670826911926
Iteration: 250, Loss: 0.6948962211608887
Iteration: 300, Loss: 0.7211440801620483
Iteration: 350, Loss: 0.7320276498794556
Iteration: 400, Loss: 0.680740237236023
Iteration: 450, Loss: 0.7084603309631348
Iteration: 500, Loss: 0.6922925710678101
Iteration: 550, Loss: 0.7268286943435669
Iteration: 600, Loss: 0.645790696144104





C:\Users\Riddhi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       980
         1.0       0.27      1.00      0.43       366

    accuracy                           0.27      1346
   macro avg       0.14      0.50      0.21      1346
weighted avg       0.07      0.27      0.12      1346



C:\Users\Riddhi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch: 0, ValF1: 0.21378504672897194
Epoch: 0, saving better model!


NameError: name 'save_model_best' is not defined